In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
# read image
path = 'tr_img_09001.jpg'
path1 = '0.jpg'

In [ ]:
list_xx = [[217,37,580,42,591,147,222,133], [221,133,565,144,574,221,216,223]]
list_xy = [[122,294,295,126,207,214,277,270], [291,469,472,303,213,221,283,287]]

In [ ]:
def bb_coordinates_xx_xy(list_xx_xy, format_ = None):
    out_list = []
    for index, list_ in enumerate(list_xx_xy):
        temp_dict = {}
        if format_ == 'xx':
            x1, x2, x3, x4, y1, y2, y3, y4 = list_
            
        elif format_ == 'xy':
            x1, y1, x2, y2, x3, y3, x4, y4 = list_
            
        else:
            raise ValueError("format either has to be 'xx' or 'xy'")
        temp_dict["x_min"] = int((x1 + x4)/2)
        temp_dict["x_max"] = int((x2 + x3)/2)
        temp_dict["y_min"] = int((y1 + y2)/2)
        temp_dict["y_max"] = int((y3 + y4)/2)
        
        #temp_dict["index"] = [x_min, x_max, y_min, y_max]
        out_list.append(temp_dict)
    return out_list

In [ ]:
bb_c_train = bb_coordinates_xx_xy(list_xx, 'xy')
bb_c_val = bb_coordinates_xx_xy(list_xy, 'xx')

In [ ]:
def annotate_image(img, all_annotations):
    for annotation in all_annotations:
        draw_bounding_box(img, annotation)

In [ ]:
def load_image(path):
    img_path = path
    img = cv2.imread(img_path)
    return img
def plot_image(img):
    plt.imshow(img)
    plt.title(img.shape)

In [ ]:
def draw_bounding_box(img, annotation):
    
    x_min, y_min = int(annotation['x_min']), int(annotation['y_min'])
    x_max, y_max = int(annotation['x_max']), int(annotation['y_max'])
    color = class_to_color(0)
    
    cv2.rectangle(img,(x_min,y_min),(x_max,y_max), color, 2)
    
def class_to_color(class_id):
    colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255),(255,100,100),
              (100,255,100),(100,100,255),(255,100,0),(255,0,100),(100,0,255),(100,100,255),(100,255,0),
              (100,255,100)]
    return colors[class_id]

In [ ]:
img = load_image(path)
annotate_image(img, bb_c_train)
plot_image(img)

In [ ]:
img = load_image(path1)
annotate_image(img, bb_c_val)
plot_image(img)

**YOLO Annotation Type Conversion**

In [ ]:
# Convert Pascal_Voc bb to Yolo
def pascal_voc_to_yolo(x_min, y_min, x_max, y_max, image_w, image_h):
    
    class_id = 0
    pw = x_max - x_min
    ph = y_max - y_min
    cx = x_min + pw/2
    cy = y_min + ph/2
    NorX = cx / image_w
    NorY = cy / image_h
    NorW = pw / image_w
    NorH = ph / image_h
    
    return (class_id, NorX, NorY, NorW, NorH)

In [ ]:
pascal_voc_to_yolo(x_min, y_min, x_max, y_max, image_w, image_h)

**Converting annotations for a folder of images**

In [ ]:
import os
images_folder = r"D:\PyTorch\Padhai\Capstone\Incidental_Scene\Text Localization\valid\images"
labels_folder = r"D:\PyTorch\Padhai\Capstone\Incidental_Scene\Text Localization\ch4_test_gt"
labels_yolo = r"D:\PyTorch\Padhai\Capstone\Incidental_Scene\Text Localization\valid\labels"

In [ ]:
for filename in os.listdir(images_folder):
    img = cv2.imread(os.path.join(images_folder, filename))
    image_h, image_w, _ = img.shape
    
    name= filename.split('.jpg')
    dest_file = os.path.join(labels_yolo,f"{name[0]}.txt")
    file = open(dest_file,'w')
    src_fullname = os.path.join(labels_folder, f'gt_{name[0]}.txt')
    src_label_file = open(src_fullname, 'r', encoding='utf-8-sig')
    Lines = src_label_file.readlines()
    bboxes = []
    for line in Lines:
        line = line.split(',')
        line = [int(x) for x in line[:8]]
        bboxes.append(line)
    bb_c = bb_coordinates_xx_xy(bboxes, 'xy')
    for bbx in bb_c:
        (class_id, NorX, NorY, NorW, NorH) = pascal_voc_to_yolo(bbx["x_min"], bbx["y_min"], bbx["x_max"], bbx["y_max"], image_w, image_h)
        file.write(str(class_id))
        file.write(' ')
        file.write(str(NorX))
        file.write(' ')
        file.write(str(NorY))
        file.write(' ')
        file.write(str(NorW))
        file.write(' ')
        file.write(str(NorH))
        file.write('\n')
    file.close()
print("Annotations converted")